## Imports

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
torch.cuda.get_device_properties(0).name

'Tesla P100-PCIE-16GB'

In [6]:
import pandas as pd

## Load data

In [2]:
leetcode_clean = pd.read_csv("../../data/leetcode-parsed/clean_leetcode_data.csv")
leetcode_clean['complexity'].value_counts()

complexity
O(n)        1538
O(nlogn)     352
O(n ^ 2)     218
O(1)         194
O(logn)      110
other         42
O(n ^ 3)      33
O(n!)          5
O(2 ^ n)       3
Name: count, dtype: int64

In [ ]:
codecomplex = pd.read_csv("../data/codecomplex/codecomplex_data.csv")
neetcode = pd.read_csv("../data/neetcode-scraped/neetcode_data.csv")
codecomplex_neetcode = pd.read_csv("../data/merges/codecomplex_neetcode/codecomplex_neetcode_merged_data.csv")
core = pd.read_csv("../data/merges/codecomplex+neetcode+leetcode_clean/full_no_exponential+factorial.csv")

In [ ]:
core['complexity'].value_counts()

complexity
O(n)        2509
O(nlogn)    1156
O(1)         970
O(n ^ 2)     907
O(logn)      776
O(n ^ 3)     603
np           497
Name: count, dtype: int64

In [ ]:
codecomplex_neetcode['complexity'].value_counts()

complexity
O(n)        972
O(nlogn)    807
O(1)        778
O(n ^ 2)    689
O(logn)     668
O(n ^ 3)    570
np          497
Name: count, dtype: int64

In [ ]:
codecomplex['complexity'].value_counts()

complexity
O(n)        834
O(nlogn)    774
O(1)        763
O(logn)     651
O(n ^ 2)    640
O(n ^ 3)    566
np          500
Name: count, dtype: int64

In [ ]:
neetcode['labels'].value_counts()

In [ ]:
codecomplex = codecomplex.drop_duplicates()

In [ ]:
codecomplex.duplicated().sum()

np.int64(0)

## Extract indices

In [118]:
labels_to_drop = {'O(n)', 'O(nlogn)', 'O(1)', 'O(n ^ 2)'}
indices_by_label = {label: leetcode_clean[leetcode_clean['complexity'] == label] for label in labels_to_drop}

In [ ]:
idxs = pd.Index(set().union(*[v.index for v in indices_by_label.values()]))

## Drop 

In [120]:
leetcode_clean.drop(index=idxs, inplace=True)
leetcode_clean['complexity'].value_counts()

complexity
O(logn)     110
other        42
O(n ^ 3)     33
O(n!)         5
O(2 ^ n)      3
Name: count, dtype: int64

## Downsample

#### Num of samples before:

In [109]:
for key, value in indices_by_label.items():
    print(f"{key}: {value.shape}")

O(nlogn): (352,)
O(n): (1538,)
O(n ^ 2): (218,)
O(1): (194,)


### Start downsampling

In [126]:
sample_size = 80

for label in indices_by_label.keys():
    indices_by_label[label] = indices_by_label[label].sample(sample_size)

#### Num of samples after:

In [127]:
for key, value in indices_by_label.items():
    print(f"{key}: {value.shape}")

O(nlogn): (80, 2)
O(n): (80, 2)
O(n ^ 2): (80, 2)
O(1): (80, 2)


## And cat the downsampled ver. back

In [128]:
downsampled = [indices_by_label[label] for label in labels_to_drop] + [leetcode_clean]
leetcode_clean = pd.concat(downsampled)

In [129]:
leetcode_clean['complexity'].value_counts()

complexity
O(logn)     110
O(nlogn)     80
O(n)         80
O(n ^ 2)     80
O(1)         80
other        42
O(n ^ 3)     33
O(n!)         5
O(2 ^ n)      3
Name: count, dtype: int64

In [ ]:
core.to_csv("../data/merges/codecomplex+neetcode+leetcode_clean/different_class_distributions/ver.3.0.csv", index=False)